In [ ]:
import os
from more_itertools import bucket
import pandas as pd

In [ ]:
def js_to_python(s):
    d = {
        "true": "True",
        "false": "False",
        "null": "None",
        "[]": "None",
        "][": "],["
    }
    for k, v in d.items():
        s = s.replace(k, v)
    return s

def read_line(s):
    try:
        a, b = eval(js_to_python(s))
        return a
    except Exception as e:
        print(e)
        return []

def listdecorator(f):
    return lambda x: list(f(x))

@listdecorator
def get_trial_data(exp_dicts):
    return filter(is_trial, exp_dicts)
        
def is_trial(d):
    st = 'stimulus_type'
    return (d[st] == 'test') if (st in d.keys()) else False

def get_all_stimuli(trials):
    return [os.path.basename(trial[s]) for trial in trials for s in ['left', 'right']]

def get_group(fn):
    for g in ['low', 'high', 'medium']:
        if g in fn:
            return g

def nonempty_vp_idx(Ns):
    return map(lambda x: x[0], (filter(lambda x: x[1] > 0, enumerate(Ns))))

In [ ]:
fn = "../data/experiment/jatos_results_20230102094455.txt"  # single vp export
fn = "../data/experiment/jatos_results_20230102094507.txt"  # export all
with open(fn) as fh:
    raw_data = [read_line(s) for s in fh]
len(raw_data)

In [ ]:
Ns = [len(s) for s in raw_data]
print(Ns)

In [ ]:
for vp in nonempty_vp_idx(Ns):
    trials = get_trial_data(raw_data[vp])

    stimulus_set = set(get_all_stimuli(trials))

    grouped_stimuli = {g: list(bucket(stimulus_set, lambda x: get_group(x))[g]) for g in ['low', 'medium', 'high']}

    losswin_counts = {}
    for g in ['low', 'high', 'medium']:
        losswin_counts[g] = {}
        for i in range(len(grouped_stimuli[g])):
            p1 = grouped_stimuli[g][i]
            for p2 in grouped_stimuli[g][(i+1):]:
                losswin_counts[g][(p1, p2)] = [0, 0]

    for trial in trials:
        left, right = [os.path.basename(trial[lr]) for lr in ['left', 'right']]
        if left == right:
            continue
        g = get_group(left)
        idx =  0 if trial['response'] == 'f' else 1
        try:
            losswin_counts[g][(left, right)][idx] += 1
        except KeyError:
            losswin_counts[g][(right, left)][idx-1] += 1

    for g in ['low', 'medium', 'high']:
        df = pd.DataFrame(losswin_counts[g]).T
        df = df.rename({0: "win1", 1: "win2"}, axis=1)
        df.to_pickle(f"../data/processed/exp_processed_vp-{vp}_g-{g}.pkl")